# QUY TRÌNH PHÂN TÍCH DỮ LIỆU NGÂN HÀNG (ROA ANALYSIS)

Đây là file Jupyter Notebook tổng hợp toàn bộ quy trình phân tích từ A-Z, bao gồm: 
1. **EDA**: Làm sạch & Khám phá dữ liệu
2. **Model Selection**: Lựa chọn mô hình tối ưu (OLS vs FEM vs REM)
3. **Robust Check**: Kiểm tra tính vững và phi tuyến
4. **Machine Learning**: Dự báo và Cây quyết định
5. **Visualization**: Trực quan hóa kết quả để báo cáo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor, plot_tree

# LƯU Ý: Bạn cần tải file Excel/CSV dữ liệu của bạn lên cùng thư mục với notebook này
file_path = 'Stata 10.05.2020_Org.xlsx - Sheet1.csv'
try:
    df = pd.read_csv(file_path)
    df = df.rename(columns={'INF': 'INFLATION'})
    print("Đã đọc dữ liệu thành công!")
except:
    print("Lỗi: Không tìm thấy file dữ liệu. Hãy đảm bảo bạn đã upload file csv/excel vào cùng thư mục.")

## BƯỚC 2: PHÂN TÍCH TƯƠNG QUAN (EDA)

In [ ]:
if 'df' in locals():
    numeric_cols = ['ROA', 'SIZE', 'NPL', 'CASH', 'DEP', 'VCSH', 'GDP', 'INFLATION']
    plt.figure(figsize=(10, 8))
    sns.heatmap(df[numeric_cols].corr(), annot=True, fmt=".2f", cmap='coolwarm')
    plt.title('Ma trận tương quan')
    plt.show()

## BƯỚC 3 & 4: MÔ HÌNH HỒI QUY (FEM ROBUST)

In [ ]:
if 'df' in locals():
    independent_vars = ['SIZE', 'NPL', 'CASH', 'DEP', 'VCSH', 'GDP', 'INFLATION']
    formula_fem = f"ROA ~ {' + '.join(independent_vars)} + C(BANK)"
    
    # Chạy FEM Robust
    model_fem = smf.ols(formula=formula_fem, data=df).fit(cov_type='cluster', cov_kwds={'groups': df['BANK']})
    print(model_fem.summary())

## BƯỚC 5: DỰ BÁO (MACHINE LEARNING)

In [ ]:
if 'df' in locals():
    data_ml = df[independent_vars + ['ROA']].dropna()
    X = data_ml[independent_vars]
    y = data_ml['ROA']
    
    model_lr = LinearRegression()
    model_lr.fit(X, y)
    
    # Kịch bản giả định
    scenarios = pd.DataFrame({
        'SIZE': [20.0, 18.0],
        'NPL': [0.01, 0.05],
        'CASH': [0.15, 0.10],
        'DEP': [0.70, 0.80],
        'VCSH': [0.10, 0.05],
        'GDP': [0.07, 0.05],
        'INFLATION': [0.04, 0.10]
    }, index=['Kịch bản Tốt', 'Kịch bản Xấu'])
    
    scenarios['ROA Dự báo (%)'] = model_lr.predict(scenarios) * 100
    print(scenarios[['ROA Dự báo (%)']])